In [1]:
import pandas as pd
import numpy as np
import pickle
from pandas import ExcelWriter
import ffn
%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

#pull in data from AlphaVantage
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='NXY0VT9AHBRYGKKC',output_format='pandas')

def historical_data(ticker, outsize = "full"):
    #outsize can be compact and full. 
    #compact returns only the latest 100 data points; full 
    #returns the full-length time series of up to 20 years of historical data. The "compact" option is 
    #recommended if you would like to reduce the data size of each API call
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={0}&apikey=NXY0VT9AHBRYGKKC&datatype=csv&outputsize={1}'.format(ticker, outsize)
    df = pd.read_csv(alphavantage_link)
    return df

In [2]:
EQ = ['MA','SPY']
Maindf = historical_data("V")
Maindf.index = pd.to_datetime(Maindf.timestamp) #set the index as the timestamp
#Maindf.index = pd.to_datetime(Maindf.index) #make the index datetime
Maindf = Maindf[['timestamp','adjusted_close']]
Maindf.columns = ["timestamp","V"]

for ticker in EQ:
    temp_df = historical_data(ticker)
    temp_df.index = pd.to_datetime(temp_df.timestamp) #set the index as the timestamp
    temp_df = temp_df[['adjusted_close']]
    temp_df.columns = [ticker]
    Maindf = Maindf.join(temp_df, how = "left")

Maindf.dropna(inplace = True)
Maindf = Maindf.sort_index()



In [3]:
del Maindf['timestamp']


In [4]:
Maindf = Maindf.loc[~(Maindf==0).all(axis=1)]

In [5]:
perf = Maindf.calc_stats()

C:\Users\scheng\AppData\Local\Continuum\anaconda3\lib\site-packages\ffn\core.py:2054: RuntimeWarning: invalid value encountered in minimum
  negative_returns = np.minimum(returns, 0.)
C:\Users\scheng\AppData\Local\Continuum\anaconda3\lib\site-packages\ffn\core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)


In [6]:
perf.display()

Stat                 V           MA          SPY
-------------------  ----------  ----------  ----------
Start                2008-03-19  2008-03-19  2008-03-19
End                  2018-07-11  2018-07-11  2018-07-11
Risk-free rate       0.00%       0.00%       0.00%

Total Return         953.41%     924.19%     164.16%
Daily Sharpe         0.91        0.86        0.57
Daily Sortino        1.53        1.44        0.89
CAGR                 25.65%      25.31%      9.88%
Max Drawdown         -51.90%     -62.66%     -51.49%
Calmar Ratio         0.49        0.40        0.19

MTD                  4.28%       3.53%       2.09%
3m                   15.50%      18.04%      5.47%
6m                   15.64%      26.62%      1.15%
YTD                  21.54%      34.83%      4.66%
1Y                   45.74%      64.52%      16.48%
3Y (ann.)            26.66%      29.59%      11.95%
5Y (ann.)            24.71%      28.45%      12.81%
10Y (ann.)           23.09%      23.55%      10.66%
Since Incep

In [7]:
perf.stats

,MA,SPY,V
start,2008-03-19 00:00:00,2008-03-19 00:00:00,2008-03-19 00:00:00
end,2018-07-11 00:00:00,2018-07-11 00:00:00,2018-07-11 00:00:00
rf,0,0,0
total_return,9.24194,1.64165,9.5341
cagr,0.253121,0.0987934,0.256544
max_drawdown,-0.626631,-0.51487,-0.519023
calmar,0.40394,0.19188,0.494283
mtd,0.0352869,0.0209009,0.0428086
three_month,0.180405,0.054727,0.154978
six_month,0.266184,0.0115439,0.156382


In [8]:
perf['MA'].display_monthly_returns()

  Year    Jan     Feb    Mar    Apr     May     Jun    Jul    Aug     Sep     Oct    Nov    Dec     YTD
------  -----  ------  -----  -----  ------  ------  -----  -----  ------  ------  -----  -----  ------
  2008   0       0      7.01  24.82   10.96  -13.97  -7.99  -0.66  -26.89  -16.57  -1.7   -1.63  -31.26
  2009  -4.91   16.39   5.98   9.63   -3.88   -5.12  16.08   4.43   -0.24    8.42   9.97   6.28   79.73
  2010  -2.32  -10.22  13.21  -2.29  -18.65   -1.11   5.35  -5.56   12.93    7.24  -1.26  -5.45  -12.22
  2011   5.6     1.71   4.64   9.66    4.04    4.98   0.68   8.73   -3.81    9.54   7.87  -0.46   66.72
  2012  -4.59   18.12   0.13   7.62  -10.12    5.81   1.57  -3.13    6.76    2.16   6.02   0.53   32.09
  2013   5.58   -0.11   4.5    2.3     3.13    0.75   6.39  -0.74   11.01    6.68   6.1    9.81   70.67
  2014  -9.3     2.7   -3.89  -1.39    3.94   -3.9    1.07   2.24   -2.49   13.47   4.23  -1.29    3.73
  2015  -4.61    9.87  -4.15   4.61    2.27    1.32   4.37  -5.1